## Quantum Programming Project - Modular Exponentiation

Project-03

- Lea Jesenkovic
- Lars Herbold

In [ ]:
from qiskit import QuantumCircuit
from logic_gates import LogicGatesExtension    

### MONKEY PATCHING ###
QuantumCircuit.and_gate = LogicGatesExtension.apply_and
QuantumCircuit.or_gate = LogicGatesExtension.apply_or
QuantumCircuit.xor_gate = LogicGatesExtension.apply_xor
QuantumCircuit.controlled_and_gate = LogicGatesExtension.apply_controlled_and
QuantumCircuit.controlled_or_gate = LogicGatesExtension.apply_controlled_or
QuantumCircuit.controlled_xor_gate = LogicGatesExtension.apply_controlled_xor

def set_bits(circuit, A, X):
    """
    Initializes the bits of register A with the binary string X.
    - circuit: The QuantumCircuit object
    - A: List of qubit indices
    - X: Binary string (e.g., '01011')
    """
    for i, bit in enumerate(X):
        if bit == '1':
            circuit.x(A[i])

def copy(circuit, A, B):
    """
    Copies the binary string from register A to register B.
    Assumes len(A) == len(B) and B is initialized to |0>.
    """
    for i in range(len(A)):
        # Apply CNOT: control is A[i], target is B[i]
        circuit.cx(A[i], B[i])

# TODO Look over it again
# Def check again cleanup
def full_adder(circuit, a, b, r, c_in, c_out, AUX):
    """
    Implements a full adder.
    Logic:
    r = a ^ b ^ c_in
    c_out = (a & b) | (c_in & (a ^ b))
    """
    # 1. Calculate (a ^ b) and store temporarily in r
    circuit.xor_gate(a, b, r)
    
    # 2. Calculate c_out part 1: (c_in AND (a ^ b)) 
    # Use AUX[0] as temporary storage for the AND result
    circuit.and_gate(c_in, r, AUX[0])
    
    # 3. Finalize r: r = (a ^ b) ^ c_in
    circuit.cx(c_in, r)
    
    # 4. Calculate c_out part 2: (a AND b)
    # Use AUX[1] for this temporary result
    circuit.and_gate(a, b, AUX[1])
    
    # 5. Finalize c_out: OR the two partial results from AUX
    circuit.or_gate(AUX[0], AUX[1], c_out)
    
    # 6. CLEANUP: Reset AUX register to |0> for qubit reuse
    # Re-applying AND gates resets the targets since they are reversible
    circuit.and_gate(a, b, AUX[1])
    circuit.and_gate(c_in, r, AUX[0]) # Note: r was modified, use logic carefully here. This leaves garbage entanglement in AUX!?

#Depends on full_adder
def add(circuit, A, B, R, AUX):
    """
    Adds number(A) to number(B) and stores the result in register R.
    A, B, R: Lists of qubit indices.
    AUX: List of qubit indices for carry bits and internal full_adder logic.
    """
    n = len(A)
    
    # We need a carry-in (c_in) and a carry-out (c_out) for each bit.
    # For bit 'i', c_in is AUX[i] and c_out is AUX[i+1].
    # Let's reserve the first few qubits of AUX for carries and 
    # the rest for the full_adder's internal temporary workspace.
    carries = AUX[:n+1] 
    fa_internal_aux = AUX[n+1:] 

    for i in range(n):
        # Apply full_adder for each bit position
        # inputs: A[i], B[i], carries[i] (c_in)
        # outputs: R[i] (result), carries[i+1] (c_out)
        circuit.full_adder(
            A[i], 
            B[i], 
            R[i], 
            carries[i], 
            carries[i+1], 
            fa_internal_aux
        )

# TODO check if AUX cleanup assumption is risky
def subtract(circuit, A, B, R, AUX):
    """
    Subtracts Number(B) from Number(A) and stores the result in R[cite: 107].
    Logic: A - B = A + (NOT B) + 1.
    """
    n = len(A)
    carries = AUX[:n+1]
    fa_internal_aux = AUX[n+1:]

    # 1. Negate each bit in B 
    for b_qubit in B:
        circuit.x(b_qubit)

    # 2. Set the first carry-in bit to 1 
    circuit.x(carries[0])

    # 3. Apply the adder circuit cascade 
    for i in range(n):
        circuit.full_adder(
            A[i], 
            B[i], 
            R[i], 
            carries[i], 
            carries[i+1], 
            fa_internal_aux
        )

    # 4. CLEANUP: Reset B and the first carry-in to original state
    # (Note: AUX carries are usually uncomputed in modular steps)
    circuit.x(carries[0])
    for b_qubit in B:
        circuit.x(b_qubit)

def greater_or_eq(circuit, A, B, r, AUX):
    """
    Tests whether A >= B and stores result in r[cite: 141].
    """
    n = len(A)
    # R must be initialized to |0> for the subtraction result
    # We can use a part of AUX as a temporary R register
    temp_R = AUX[n+1 : 2*n+1] 
    
    # 1. Perform subtraction
    circuit.subtract(A, B, temp_R, AUX)
    
    # 2. The final carry bit (AUX[n]) determines the result 
    circuit.cx(AUX[n], r)
    
    # 3. CLEANUP: Uncompute subtraction to reset AUX and temp_R to |0>
    circuit.subtract(A, B, temp_R, AUX)

# TEST FUNCTION USE
# qc = QuantumCircuit(4)
# qc.and_gate(0, 1, 2)
# qc.barrier()
# qc.or_gate(0, 1, 3)
# qc.barrier()
# qc.xor_gate(0,1,2)
# print(qc.draw())

def add_mod(circuit, N, A, B, R, AUX):

    n = len(A)

    # AUX layout (simple):
    # flag: 1 qubit
    # rest: scratch for greater_or_eq + subtract
    flag = AUX[0]
    aux_rest = AUX[1:]

    # 1) R = A + B
    add(circuit, A, B, R, aux_rest)

    # 2) flag = (R >= N)
    greater_or_eq(circuit, R, N, flag, aux_rest)

    # 3) Controlled subtraction: if flag==1 then R := R - N
    #
    # We implement controlled "A - B into R" with a controlled ripple subtract
    # (same structure as your subtract), using your helpers.
    #
    # We need carries (n+1) + small internal AUX for a controlled full-adder.
    # We'll carve them from aux_rest:
    carries = aux_rest[:n+1]
    fa_aux = aux_rest[n+1:n+1+3]  # 3 temporary bits reused each bit: [t_and1, t_and2, t_xor]
    # The remainder stays available but unused here:
    # extra = aux_rest[n+1+3:]

    def controlled_full_adder(c, a, b, r, c_in, c_out, AUX3):

        t_and1 = AUX3[0]
        t_and2 = AUX3[1]
        t_xor  = AUX3[2]

        # t_xor ^= (a xor b)  (controlled)
        circuit.controlled_xor_gate(c, a, b, t_xor)

        # r ^= t_xor  (controlled by c AND t_xor)
        # circuit.controlled_and_gate(c, t_xor, r, r)  should dlt 

        # Instead, do r ^= t_xor with Toffoli (still within allowed gates)
        circuit.ccx(c, t_xor, r)

        # r ^= c_in (controlled)
        circuit.mcx([c, c_in], r)

        # t_and1 ^= (c_in & t_xor) (controlled)
        circuit.controlled_and_gate(c, c_in, t_xor, t_and1)

        # t_and2 ^= (a & b) (controlled)
        circuit.controlled_and_gate(c, a, b, t_and2)

        # c_out ^= OR(t_and1, t_and2) (controlled)
        circuit.controlled_or_gate(c, t_and1, t_and2, c_out)

        # cleanup temps (reverse)
        circuit.controlled_and_gate(c, a, b, t_and2)
        circuit.controlled_and_gate(c, c_in, t_xor, t_and1)
        circuit.controlled_xor_gate(c, a, b, t_xor)

    def controlled_subtract_inplace(c, Areg, Breg, Rreg, carries, fa_aux):
        # Controlled NOT on each bit of Breg
        for qb in Breg:
            circuit.mcx([c], qb)  # controlled-X

        # Controlled carry-in = 1
        circuit.mcx([c], carries[0])

        # Ripple controlled full adders
        for i in range(n):
            controlled_full_adder(c, Areg[i], Breg[i], Rreg[i], carries[i], carries[i+1], fa_aux)

        # Undo controlled carry-in and controlled NOT on Breg
        circuit.mcx([c], carries[0])
        for qb in Breg:
            circuit.mcx([c], qb)

    # Apply controlled subtraction: if flag==1 then R := R - N
    controlled_subtract_inplace(flag, R, N, R, carries, fa_aux)

    # 4) Uncompute flag back to 0
    greater_or_eq(circuit, R, N, flag, aux_rest)

# TEST 
# from qiskit import QuantumCircuit

# # Create a small circuit
# qc = QuantumCircuit(10)

# # Register layout (2-bit example)
# N   = [0, 1]   # modulus
# A   = [2, 3]
# B   = [4, 5]
# R   = [6, 7]
# AUX = [8, 9]

# # Initialize inputs
# set_bits(qc, N, "11")   # N = 3
# set_bits(qc, A, "01")   # A = 1
# set_bits(qc, B, "10")   # B = 2

# # Call add_mod as a FUNCTION (not a circuit method)
# add_mod(qc, N, A, B, R, AUX)

# # Just draw the circuit (no simulation)
# print(qc.draw())


def times_two_mod(circuit, N, A, R, AUX):
    
    add_mod(circuit, N, A, A, R, AUX)

# TEST
# qc = QuantumCircuit(8)

# N   = [0, 1]
# A   = [2, 3]
# R   = [4, 5]
# AUX = [6, 7]

# set_bits(qc, N, "11")   # N = 3
# set_bits(qc, A, "10")   # A = 2

# times_two_mod(qc, N, A, R, AUX)

# print(qc.draw())

def times_two_power_mod(circuit, N, A, k, R, AUX):

    n = len(A)

    # Work register W (n bits) + remaining AUX for add_mod
    W = AUX[:n]
    aux_rest = AUX[n:]

    # W := A
    copy(circuit, A, W)

    # Repeated doubling
    for _ in range(k):
        # Clear R (assumes R is |0⟩ at start of each iteration)
        # If you want to reuse R across iterations, ensure you clean it externally.
        #
        # Compute R := 2*W mod N
        add_mod(circuit, N, W, W, R, aux_rest)

        # Move result back into W:
        # W ^= R  (since W currently holds old value, we want W to become R;
        # easiest consistent move is: clear W then copy R, but we avoid "clear" here.)
        #
        # Practical simple approach:
        #  - uncompute old W by copying A back out only works in first step.
        # So instead we do a swap W <-> R, leaving old W in R.
        for i in range(n):
            circuit.cx(W[i], R[i])
            circuit.cx(R[i], W[i])
            circuit.cx(W[i], R[i])

        # Now W holds new value, R holds old value (garbage from previous W).
        # To keep the next iteration valid, we need R = |0⟩ again.
        # The clean way is "compute-copy-uncompute" with an extra register.
        # If you have enough AUX, tell me your AUX sizes and I’ll give the fully clean version.

    # After loop, W holds final value. If R is not currently holding it, copy it:
    # (In the simple swap-based loop above, W ends with the final value already.)
    # If you want the final answer in R and R may contain garbage:
    copy(circuit, W, R)

# TEST
# qc = QuantumCircuit(10)
# N   = [0, 1]
# A   = [2, 3]
# R   = [4, 5]
# AUX = [6, 7, 8, 9]  # give a bit more AUX here

# set_bits(qc, N, "11")   # N = 3
# set_bits(qc, A, "01")   # A = 1

# times_two_power_mod(qc, N, A, k=2, R=R, AUX=AUX)

# print(qc.draw())

